What is the SIR model

SIR model are mathematical models used to simulate the spread of a disease in a population, based on the assumption that people can be in one out of three compartments at time t: Susceptible (S), Infected (I) and Recovered (R). The total amount of people in the system are constant at all times - meaning that no people leave or enter the system. 

We have chosen to simulate the disease dynamics of 1000 people, initially with 1 infected individual, 999 susceptible and 0 recovered. In the specific case of a basic SIR model, people can only move from S to I and from I to R, meaning that the system will reach steady states either when all people have reached recovered state or if there are no infected people left.

As the model is stochastic and based on probabilities, runs might vary even though the parameters are the same. Therefore, for every parameter combination, the simulation is run 100 times, and different statistics are obtained, for example the mean number of infected of these 100 runs. 

In [20]:
# Initialization
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from tabulate import tabulate

In [81]:
prob_S_I_list = [0.2, 0.5, 0.8]
prob_I_R_list = [0.2, 0.5, 0.8] 

n_runs = 100
time_break = 30

statistics = np.array([np.zeros(5)])
# First parameter value
for prob1 in prob_S_I_list:
    # Second parameter value
    for prob2 in prob_I_R_list:
        # Run n simulations
        p_vector = np.array([])
        I_max = np.array([])
        I_max_t = np.array([])
        for i in range(n_runs):
            time_counter = 0 
            n = 1000

            # Vector of states
            states = np.repeat("S",n)
            states[0] = "I"

            # Probabilities of entering 
            prob_S_I = prob1
            prob_I_R = prob2

            n_S = np.array([len(np.where(states == "S")[0])])
            n_I = np.array([len(np.where(states == "I")[0])])
            n_R = np.array([len(np.where(states == "R")[0])])


            while ("I" in states):

                # Get positions of S, I, R
                S_index = np.where(states == "S")[0]
                I_index = np.where(states == "I")[0]
                R_index = np.where(states == "R")[0]

                # Update susceptible
                prob_S_I_t = prob_S_I * len(I_index)/n
                states[S_index] = np.random.choice(np.array(["S","I"]),p = np.array([1-prob_S_I_t, prob_S_I_t]), size = len(S_index))


                # Update infected
                states[I_index] = np.random.choice(np.array(["I", "R"]),p = np.array([1-prob_I_R, prob_I_R]), size = len(I_index))


                n_S = np.concatenate([n_S, np.array([len(np.where(states == "S")[0])])])
                n_I = np.concatenate([n_I, np.array([len(np.where(states == "I")[0])])])
                n_R = np.concatenate([n_R, np.array([len(np.where(states == "R")[0])])])

                time_counter += 1
                
                
            # Save statistics
            I_max = np.concatenate([I_max, np.array([np.max(n_I)])])
            I_max_t = np.concatenate([I_max_t, np.array([np.argmax(n_I, axis=0)])])
            #I_max_t = np.argmax(n_I, axis=0)
            
            if len(n_I) > time_break:
                p_vector = np.concatenate([p_vector, np.array([1])])
            else:
                p_vector = np.concatenate([p_vector, np.array([0])])
        
        # Analyse statistics
        I_max_mean = np.mean(I_max)
        I_max_sd = np.std(I_max)
        
        I_max_t_mean = np.mean(I_max_t)
        I_max_t_sd = np.std(I_max_t)
    
        print(type(I_max))
        
        conf = stats.norm.interval(alpha=0.95, loc=np.mean(I_max), scale=stats.sem(I_max))
        
        
        new_stats = (prob_S_I, prob_I_R, I_max_mean, np.round(conf,2), I_max_t_mean)
        print(new_stats)
        statistics = np.vstack((statistics, new_stats))
        
        #print("Maximum infected")
        #print("mean: ", I_max_mean, "CI: ", conf)
        #print("sd: ", np.round(I_max_sd,2))
        #print("----------------------")

        #likelihood = np.sum(p_vector)/len(p_vector)
        
        #print("The likelihood that the disease dies out after {} days: {}".format(time_break,np.round(1-likelihood,3)))
        


<class 'numpy.ndarray'>
(0.2, 0.2, 3.58, array([2.41, 4.75]), 6.1)
<class 'numpy.ndarray'>
(0.2, 0.5, 1.26, array([1.17, 1.35]), 0.46)
<class 'numpy.ndarray'>
(0.2, 0.8, 1.08, array([1.01, 1.15]), 0.07)
<class 'numpy.ndarray'>
(0.5, 0.2, 176.44, array([152.91, 199.97]), 19.07)
<class 'numpy.ndarray'>
(0.5, 0.5, 3.32, array([2.26, 4.38]), 3.29)
<class 'numpy.ndarray'>
(0.5, 0.8, 1.38, array([1.22, 1.54]), 0.43)
<class 'numpy.ndarray'>
(0.8, 0.2, 383.31, array([351.39, 415.23]), 14.31)
<class 'numpy.ndarray'>
(0.8, 0.5, 53.47, array([44.13, 62.81]), 12.46)
<class 'numpy.ndarray'>
(0.8, 0.8, 2.53, array([1.79, 3.27]), 1.83)


In [ ]:
plt.figure()
plt.plot(n_S, "-b", label = "Susceptible")
plt.plot(n_I, "-r", label = "Infected")
plt.plot(n_R, "-g", label = "Recovered")
plt.ylabel("Number of people")
plt.xlabel("Days")
plt.legend()
plt.show()

print("Maximum number of infected: ", max(n_I))

In [ ]:
### Results


display(Latex(str(3)))

{{a}}

The statistical results of the {{n_runs}} runs are shown below. This includes the tested probabilities (S --> I and I --> R), the maximum number of infected and the time at which this happens. Furthermore the confidence intervals for the mean of the maximum number of infected is shown. 

In [79]:
print(tabulate(statistics[1:,], headers=["S --> I","I --> R", "I max mean", "CI",  "time at I max"], tablefmt='orgtbl'))

|   S --> I |   I --> R |   I max mean | CI              |   time at I max |
|-----------+-----------+--------------+-----------------+-----------------|
|       0.2 |       0.2 |         3.42 | [2.55 4.29]     |            6.24 |
|       0.2 |       0.5 |         1.27 | [1.15 1.39]     |            0.5  |
|       0.2 |       0.8 |         1.03 | [1.   1.06]     |            0.05 |
|       0.5 |       0.2 |       169.14 | [145.14 193.14] |           18.15 |
|       0.5 |       0.5 |         3.33 | [2.27 4.39]     |            3.13 |
|       0.5 |       0.8 |         1.47 | [1.16 1.78]     |            0.6  |
|       0.8 |       0.2 |       353.44 | [317.49 389.39] |           13.19 |
|       0.8 |       0.5 |        37.1  | [28.18 46.02]   |            8.89 |
|       0.8 |       0.8 |         3.13 | [2.28 3.98]     |            1.75 |


As can be seen from the results above, the largest mean of infected is obtained when the probability of going from S to I is relatively high (0.5 or 0.8) and the probability of recovering (0.2) is low. This is realistic, as these scenarios will result in a lot of infected people for a long time, and few people recovering. This means that the infected people have a longer infectious period. 

In [77]:
np.argmax([100,3,4])

0